# Diagnóstico por persona

En esta "V1" se mejora de la "V0" porque tiene en cuenta discapacidad como diagnóstico y se exporta el dataframe son columnas con multi-index.

In [1]:
# Librerías usadas:
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [2]:
# Levanto el stock existente en una carpeta:

ruta = '.\Stocks\\'
contenido = os.listdir(ruta)

print("Inicio del proceso: " + datetime.now().strftime('%H:%M:%S'))

# Dataframe con el stock:
dfs = pd.DataFrame()

archivos = []
for fichero in contenido:
    if os.path.isfile(os.path.join(ruta, fichero)) and fichero.endswith('.xlsx'):
        df_temp = pd.read_excel(ruta+fichero)
        dfs = pd.concat([dfs, df_temp], axis=0)
        print("Se cargó el archivo: ", fichero, "a las: " + datetime.now().strftime('%H:%M:%S'))

df_temp = []
print("Fin de la carga de archivos")        


Inicio del proceso: 14:18:32
Se cargó el archivo:  Stock_202009.xlsx a las: 14:21:07
Se cargó el archivo:  Stock_202010.xlsx a las: 14:23:59
Se cargó el archivo:  Stock_202011.xlsx a las: 14:27:13
Se cargó el archivo:  Stock_202012.xlsx a las: 14:30:39
Se cargó el archivo:  Stock_202101_SanCor.xlsx a las: 14:34:02
Se cargó el archivo:  Stock_202102_SanCor.xlsx a las: 14:37:00
Se cargó el archivo:  Stock_202103_SanCor.xlsx a las: 14:40:10
Se cargó el archivo:  Stock_202104_SanCor.xlsx a las: 14:43:22
Se cargó el archivo:  Stock_202105_SanCor.xlsx a las: 14:46:39
Se cargó el archivo:  Stock_202106_SanCor.xlsx a las: 14:50:05
Se cargó el archivo:  Stock_202107_SanCor.xlsx a las: 14:53:34
Se cargó el archivo:  Stock_202108_SanCor.xlsx a las: 14:57:07
Se cargó el archivo:  Stock_202109_SanCor.xlsx a las: 15:00:40
Fin de la carga de archivos


## 1.2- Renombrar columnas

In [6]:
# Se pueden renombrar incluso columnas que no existen (mejorar la eficiencia en función del warning)
dfs.rename( columns = {    'Periodo ID': 'Periodo',
                           'Persona ID': 'Persona',
                           'Persona Fecha Nacimiento': 'Fecha_nac',
                           'Discapacitado ID': 'Discapacidad',
                           'Marca Persona con Diabetes DESC': 'Diabetes',
                           'Marca Persona con Hepatitis DESC': 'Hepatitis',
                           'Marca Persona con PMI DESC': 'PMI',
                           'Marca Persona con Tratamiento Cronico DESC': 'Cronico',
                           'Marca Persona con Tratamiento Oncológico DESC':  'Oncologico',
                           'Sexo ID': 'Sexo',
                           'Plan Sin Segmento ID': 'Plan',
                           'Posicion Asociado DESC': 'Posicion Asociado',
                           'Zona de Promocion DESC': 'Zona de Promocion',
                           'Segmento Agrupado DESC': 'Segmento',
                           'Plan Agrupado Sin Segmento DESC': 'Plan Agrupado',
                           'Stock Asociados Salud': 'Stock',
                           'Edad Periodo ID': 'Edad',
                           'ACE Stock Asociados Salud': 'ACE',
                           'Consumos Costo con Subsidios SSS': 'Consumo3',
                           'Recaudacion Costo con Otros Subsidios': 'Recaudacion3'                      
                           }, inplace=True)

In [7]:
len(dfs)

7738496

In [8]:
dfs.Stock.sum()

7740092

# 2- Cálculo de marca GC

Pasos a seguir:<br>
1- Buscar el consumo por persona<br>
2- Definir los rangos para marcara a cada persona<br>
3- Marcar los GC por importe<br>
4- Calcular consumo anualizado<br>
5- Marcar los rangos de consumos según las definiciones iniciales<br>
6- Cruzar con el stock su marca<br>
7- Para las personas que no estan en consumos o stock marcar "no corresponde"<br>

In [14]:
# Carpeta en que se encuentran las auxiliares:
ruta_aux =  '.\Auxiliares\\'

# Guardo cantidad de registros iniciales para comparar al final:
len_dfs = len(dfs)

In [15]:
# Seteo de variables:

# Defino importe gran consumidor
importe_gc = 600000

# De cada rango se define su límite inferior
c_muy_alto = 600 * 1000
c_alto = 100 * 1000
c_medio = 10 * 1000
c_bajo = 200 * 12 # 200 pesos por mes por el efecto cápita
# El consumo c_muy_bajo queda por defecto.


In [11]:
# Levanto el archivo .xlsx de la carpeta:

ruta = '.\Consumos\\'
contenido = os.listdir(ruta)

print("Inicio del proceso: " + datetime.now().strftime('%H:%M:%S'))

# Dataframe con el stock:
df_consumos = pd.DataFrame()

archivos = []
for fichero in contenido:
    if os.path.isfile(os.path.join(ruta, fichero)) and fichero.endswith('.xlsx'):
        df_temp = pd.read_excel(ruta+fichero)
        df_consumos = pd.concat([df_consumos, df_temp], axis=0)
        print("Se cargó el archivo: ", fichero, "a las: " + datetime.now().strftime('%H:%M:%S'))

df_temp = []
print("Fin de la carga de archivos")


Inicio del proceso: 15:20:26
Se cargó el archivo:  30.1 CSR por persona 202009-202109.xlsx a las: 15:21:30
Fin de la carga de archivos


In [12]:
# Renombro columnas:
# Se pueden renombrar incluso columnas que no existen (mejorar la eficiencia en función del warning)

df_consumos.rename( columns = 
                      {    'Periodo ID': 'Periodo',
                           'Persona ID': 'Persona',
                           'Persona Fecha Nacimiento': 'Fecha_nac',
                           'Discapacitado ID': 'Discapacidad',
                           'Marca Persona con Diabetes DESC': 'Diabetes',
                           'Marca Persona con Hepatitis DESC': 'Hepatitis',
                           'Marca Persona con PMI DESC': 'PMI',
                           'Marca Persona con Tratamiento Cronico DESC': 'Cronico',
                           'Marca Persona con Tratamiento Oncológico DESC':  'Oncologico',
                           'Sexo ID': 'Sexo',
                           'Plan Sin Segmento ID': 'Plan',
                           'Posicion Asociado DESC': 'Posicion Asociado',
                           'Zona de Promocion DESC': 'Zona de Promocion',
                           'Segmento Agrupado DESC': 'Segmento',
                           'Plan Agrupado Sin Segmento DESC': 'Plan Agrupado',
                           'Stock Asociados Salud': 'Stock',
                           'Edad Periodo ID': 'Edad',
                           'ACE Stock Asociados Salud': 'ACE',
                           'Consumos Costo con Subsidios SSS': 'Consumo3',
                           'Recaudacion Costo con Otros Subsidios': 'Recaudacion3'                      
                           }, inplace=True)


In [13]:

######## MARCO GRAN CONSUMIDOR:
# https://es.stackoverflow.com/questions/193267/c%C3%B3mo-creo-una-columna-en-un-dataframe-dependiendo-del-valor-de-otras-dos-column
df_consumos['GC'] = np.where(df_consumos['Consumo3'] > importe_gc, 'GC','NO GC')


######## MARCO RANGO CONSUMOS:
# Cálculo de consumo anualizado para rango consumos:
df_consumos['Cons_anualizado'] = (df_consumos['Consumo3']*12)/df_consumos['Stock']

# Como existen personas con stock == 0 -> reemplazo los nan con el valor de la columna "Consumo3"
# No encontré una función -> hago una reimputación manual
for i in range(len(df_consumos)):
    if np.isnan(df_consumos["Cons_anualizado"][i]) == True:
        df_consumos["Cons_anualizado"][i] = df_consumos["Consumo3"][i]

# Lista de condiciones para selección
# https://www.delftstack.com/es/howto/python-pandas/how-to-create-dataframe-column-based-on-given-condition-in-pandas/#lista-de-comprensi%25C3%25B3n-para-crear-nuevas-columnas-dataframe-basadas-en-una-condici%25C3%25B3n-dada-en-pandas
# Método np.select()
condiciones = [
    (df_consumos['Cons_anualizado'] >= c_muy_alto) ,
    (df_consumos['Cons_anualizado'] >= c_alto) ,
    (df_consumos['Cons_anualizado'] >= c_medio) ,
    (df_consumos['Cons_anualizado'] >= c_bajo) ,
    (df_consumos['Cons_anualizado'] < c_bajo)]

# Lista de resultados en función de las selecciones
valores_condic = ['Muy Alto', 'Alto', 'Medio', 'Bajo', 'Muy Bajo']     

# Generación de la columna calculada
df_consumos['Rango_Cons'] = np.select(condiciones, valores_condic, default='Sin valor')


######## PEGAMOS LA INFORMACION AL df DE STOCK:
dfs = pd.merge(left = dfs, right = df_consumos[["Persona","GC","Rango_Cons"]], left_on='Persona', right_on='Persona', how = "left")
# verifico cantidad de registros:
len_dfs_cons = len(dfs)
len_dfs_cons - len_dfs

# Elimino el df auxiliar por memoria:
#df_consumos = []

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0

# 3- Calcular marcas de diagnóstico

El procedimiento a seguir debe ser el siguiente:<br>
1- Si el 100% del tiempo tuvo un diagnóstico va ese. <br>
2- Si más del 60% tuvo algún diagnóstico va ese.<br>
3- Si no supera 60% en algún clúster se lo define como su peor situación (con diagnóstico).<br>
4- (solamente para consumos) = si no se encuentra la persona en el tiempo de stock analizado se lo marca como "No Corresponde".<br>

In [16]:
# Definiciones:
# Proporción mínima de meses con diagnóstico para ser considerado con algún diagnóstico:
prop_limite = 0.4
# Se puede agregar los campos considerados como diag..

# Reemplazo valores de filas con replace:
dfs['Diabetes'] = dfs['Diabetes'].replace(['No Diabético', 'Diabético'], [0, 1])
dfs['Hepatitis'] = dfs['Hepatitis'].replace(['Sin Hepatitis', 'Con Hepatitis'], [0, 1])
dfs['PMI'] = dfs['PMI'].replace(['NO', 'SI'], [0, 1])
dfs['Cronico'] = dfs['Cronico'].replace(['NO', 'SI'], [0, 1])
dfs['Oncologico'] = dfs['Oncologico'].replace(['Sin Marca', 'Paciente Oncológico'], [0, 1])
dfs['GC'] = dfs['GC'].replace(['NO GC', 'GC'], [0, 1])
dfs['Discapacidad'] = dfs['Discapacidad'].replace(['N', 'S'], [0, 1])

# Creo una columna de resultado que marque 1 si se tiene cualquier diagnostico
# Se tendría que mejorar para generalizar:
suma = (dfs['Diabetes'] + dfs['Hepatitis'] + dfs['PMI'] + 
        dfs['Cronico'] + dfs['Oncologico'] + dfs['GC'] + dfs['Discapacidad'])
dfs['algun_diag'] = np.where( suma > 0, 1, 0)

# Creo una auxiliar que resuma por persona el stock y cantidad de veces con diag.
# Agrupo por persona las diferentes métricas en otro df para luego cruzar con la base original
filas = ['Persona','Rango_Cons']
columnas = ['Stock','Diabetes', 'Hepatitis', 'PMI','Cronico', 'Oncologico', 'GC', 'Discapacidad', 'algun_diag']
aux_diag = pd.DataFrame(dfs.pivot_table(columnas , filas, aggfunc = np.sum))


# Genero una columna que calcule el % de veces que se tuvo algún diagnóstico:
aux_diag['prop_diag'] = aux_diag['algun_diag'] / aux_diag['Stock']
aux_diag.tail()

# Creamos la marca a cada persona
# Analizando las diferentes posibilidades llegué a la conclusión que si la proporción es mayor a 40% se debe poner "Con Diagnóstico"
aux_diag['Marca_Diagnostico'] = np.where(aux_diag['prop_diag'] > prop_limite, 'Con Diagnostico','Sin Diagnostico')

# Reseteo los multi index:
aux_diag = aux_diag.reset_index()

In [17]:
# lo que se acorta el df tiene relacion con personas duplicadas.
# en el caso de stock continúa ok.
len(aux_diag)

766227

In [18]:
aux_diag.Stock.sum()

7740092

# Exporto la info a csv

In [20]:
aux_diag.to_csv("Diagnosticos.csv", encoding = 'latin1')

# Todo lo siguiente no se corre en este caso

# 4- Pegar las marcas al consumo

Se hace para tener la info de los que están en el consumo y los que están en stock

In [ ]:
# Diagnósticos a pegar:
diags = ['Cronico', 'Diabetes', 'Hepatitis', 'Oncologico', 'PMI', 'algun_diag', 'prop_diag', 'Marca_Diagnostico']


# Pegamos la información de diagnósticos en consumos 12M:
df_consumos = pd.merge(left = df_consumos, right = aux_diag[diags], left_on='Persona', 
               right_on='Persona', how = "left")

In [ ]:
# Reemplazar GC = S por 1 y N por 0
df_consumos['GC'] = df_consumos['GC'].replace(['NO GC', 'GC'], [0, 1])

# 5- Resolvemos valores faltantes?

In [ ]:
#

# 6- Exportar la información en ".csv"

In [ ]:
# Exporto la información en csv:
df_consumos.to_csv("Diagnosticos.csv", encoding = 'latin1')

# 7- Levantamos csv para observar el dataframe

In [ ]:
df = pd.read_csv('Diagnosticos.csv')

In [ ]:
df.head()

In [ ]:
df.Cronico.unique()